In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta, timezone
import time
import nvda_config
import twelvedata as td
from twelvedata import TDClient


In [4]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [9]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()

# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000 

In [37]:


# Initialize a counter for the number of requests
request_count = 0

while request_count < 390:
    # Fetch time series data and indicators
    price_pdf = td.price(
        symbol=symbol
    )
    # Convert to pandas Dataframes
    price_json = price_pdf.as_json()
    price_df = pd.DataFrame(price_json, index=["price"])

    # Prepare for database insertion
    time_series_data = []

    # Process and align data
    for index, row in price_df.iterrows():
        # Generate the current datetime string
        datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        time_series_data.append(
            (
                symbol,
                datetime_str,
                row["price"]
            )
        )

    # Insert tick data into MySQL
    sql_time_series = """
    INSERT INTO nvda_new (symbol, datetime, price)
    VALUES (%s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)
    mydb.commit()

    # Increment the request counter
    request_count += 1
    print("Requests elapsed: ", request_count, " at time:", datetime.now())
    # Sleep for a short period before fetching the next data
    time.sleep(59.75)  # Adjust the sleep time as needed

print("Program completed after 390 requests.")


Requests elapsed:  4  at time: 2024-08-04 22:03:02.680491
Requests elapsed:  4  at time: 2024-08-04 22:03:02.680491


In [5]:

def get_earnings_data(ticker, outputsize=50,  start_date = None, end_date = None):
    """Fetches earnings data for a given stock symbol from Twelve Data.

    Args:
        symbol (str): The stock symbol (e.g., "AAPL", "NVDA").
        outputsize (int, optional): Number of earnings records to retrieve (default=50, max=5000).
        timeframe (str, optional): 

    Returns:
        pd.DataFrame: A pandas DataFrame containing the earnings data.
    """

    res = td.get_earnings(
        symbol=ticker,
        outputsize=outputsize, 
        start_date = start_date, 
        end_date = end_date

    )

    # Convert the response data to a Pandas DataFrame
    df = pd.DataFrame(res.as_json())

    return df


In [6]:
# Date range for historical data (funky start date due to downloading first month as test batch)
end_date = datetime.now()
start_date = end_date - timedelta(days=390)  
nvda_earnings_1year = get_earnings_data("NVDA", 1000, start_date, end_date)

print(nvda_earnings_1year)

         date               time  eps_estimate  eps_actual  difference  surprise_prc
0  2024-06-02     Transfer Agent          5.59        6.12        0.53          9.48
1  2024-05-23     Transfer Agent          0.56        0.61        0.05          8.93
2  2024-03-18  Time Not Supplied          4.63        5.16        0.53         11.45
3  2024-02-15     Transfer Agent          4.63        5.16        0.53         11.45
4  2024-01-17     Transfer Agent          3.37        4.02        0.65         19.29
5  2024-01-08     Transfer Agent          3.37        4.02        0.65         19.29
6  2023-11-21     Transfer Agent          3.37        4.02        0.65         19.29
7  2023-08-23     Transfer Agent          2.09        2.70        0.61         29.19
         date               time  eps_estimate  eps_actual  difference  surprise_prc
0  2024-06-02     Transfer Agent          5.59        6.12        0.53          9.48
1  2024-05-23     Transfer Agent          0.56        0.61       

In [10]:

nvda_earnings_1year.to_csv(symbol + '_earnings.csv', index = False)